# Отток клиентов

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Оглавление

1. [Подготовка данных](#step_1)
2. [Исследование задачи](#step_2)
3. [Борьба с дисбалансом](#step_3)
4. [Тестирование модели](#step_4)
5. [Чек-лист](#checklist)

## 1. Подготовка данных <a id="step_1"></a>

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.utils import shuffle

In [2]:
#data = pd.read_csv('...')

In [3]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Пропуски данных имеются только в столбце 'Tenure'.  
Посмотрим на распределение количественных значений:

In [5]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Заменим пропущенные значения Tenure на "0":

In [6]:
data['Tenure'] = data['Tenure'].fillna(0)

Преобразуем тип данных в некоторых столбцах:

In [7]:
data['Tenure'] = data['Tenure'].astype('int64')

In [8]:
data['HasCrCard'] = data['HasCrCard'].astype('bool')

In [9]:
data['IsActiveMember'] = data['IsActiveMember'].astype('bool')

In [10]:
data['Exited'] = data['Exited'].astype('bool')

Проверим дублированные строки:

In [11]:
data.duplicated().sum()

0

Посмотрим сколько имеется уникальных фамилий клиентов:

In [12]:
len(data['Surname'].unique())

2932

Различных фамилий слишком много, к тому же, вряд ли они могут влиять на наши данные. Исключим столбец с фамилиями, id, номерами строк из датафрейма:

In [13]:
data = data.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1)

Заменим категориальные признаки количественными методом OHE:

In [14]:
data_ohe = pd.get_dummies(data, drop_first=True)
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,True,True,101348.88,True,0,0,0
1,608,41,1,83807.86,1,False,True,112542.58,False,0,1,0
2,502,42,8,159660.80,3,True,False,113931.57,True,0,0,0
3,699,39,1,0.00,2,False,False,93826.63,False,0,0,0
4,850,43,2,125510.82,1,True,True,79084.10,False,0,1,0


Создадим переменные для признаков и целевых признаков:

In [29]:
features = data_ohe.drop('Exited', axis=1)

In [30]:
target = data_ohe['Exited']

Разделяем датасет на 3 части: тренировочная, валидационная, тестовая выборки в соотношении 3-1-1:

In [17]:
features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [18]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.5, random_state=12345)

Проведем настройку масштабирования на тренировочной выборке и применим масштабирование для всех трех выборок:

In [19]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [20]:
scaler = StandardScaler()

In [21]:
scaler.fit(features_train[numeric])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [22]:
features_train[numeric] = scaler.transform(features_train[numeric])

Проверяем размерность полученных таблиц:

In [23]:
features_train.shape

(6000, 11)

In [24]:
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [25]:
features_valid.shape

(2000, 11)

In [26]:
features_test[numeric] = scaler.transform(features_test[numeric])

In [27]:
features_test.shape

(2000, 11)

In [28]:
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(6000,)
(2000,)
(2000,)


### Вывод

На данном этапе была выполнена загрузка и предобработка данных. Было выяснено, что пропуски имеются в одном столбце 'tenure' - они были заменены нулями. Оставшиеся столбцы пропусков не имеют, а количественные значения вопросов не вызывают. Из таблицы были удалены ненужные столбцы с номерами строк, id клиента, фамилиями, т.к. они не должны оказать на модель большое позитивное влияние, а могут лишь привести к искажению предсказаний. Категориальные признаки были заменены методом one-hot-encoding. Далее выборки были разделены на тренировочную, валидационную и тестовую в соотношении 3-1-1, scaler настроен на тренировочной выборке и все три набора данных масштабированы.

## 2. Исследование задачи <a id="step_2"></a>

Рассмотрим баланс классов для выборок целевых признаков:

In [31]:
target_train.value_counts(normalize=True)

False    0.800667
True     0.199333
Name: Exited, dtype: float64

In [32]:
target_valid.value_counts(normalize=True)

False    0.791
True     0.209
Name: Exited, dtype: float64

In [33]:
target_test.value_counts(normalize=True)

False    0.7885
True     0.2115
Name: Exited, dtype: float64

Соотношение между "0" и "1" везеде примерно 80/20% соответственно.

Создадим таблицу, в которую будем записывать результаты расчетов метрик, полученные для различных моделей:

In [34]:
model_results = pd.DataFrame(columns=['model_name', 'F1-score', 'AUC-ROC'])

Создадим и обучим модели на данных "как есть" без учета дисбаланса классов. Переберем некоторые параметры моделей, используя метрику F1, AUC-ROC.

- Логистическая регрессия

Используем стандартные параметры:

In [35]:
model = LogisticRegression(random_state=123, solver='liblinear') #создаем модель
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid) #рассчитываем F1-score
probabilities_valid = model.predict_proba(features_valid) #рассчитываем AUC-ROC
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

i=0 #счетчик строк при добавлении их в таблицу
model_results.loc[i, 'model_name'] = 'Logistic Regression Default' #заполняем таблицу с результатами метрик
model_results.loc[i, 'F1-score'] = f1_score(target_valid, predictions_valid)
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1 score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1 score: 0.33390
AUC-ROC: 0.75863


Устанаваливаем 'Penalty' равным 'l1':

In [36]:
model = LogisticRegression(random_state=123, penalty='l1', solver='liblinear')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1 score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1 score: 0.33390
AUC-ROC: 0.75871


Перебираем 'solver':

In [37]:
solver_names = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
best_model = None
best_score = 0
for solver in solver_names:
    model = LogisticRegression(random_state=123, solver=solver, max_iter=4000)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid)
    print('Solver: ', solver, '; ', 'F1-score: {:.5f}'.format(f1), sep='')

Solver: newton-cg; F1-score: 0.33390
Solver: lbfgs; F1-score: 0.33390
Solver: liblinear; F1-score: 0.33390
Solver: sag; F1-score: 0.33390
Solver: saga; F1-score: 0.33390


Перебираем параметр регуляризации 'С':

In [38]:
best_model = None
best_score = 0
C_reg = 0.01
while C_reg < 1.01:
    model = LogisticRegression(random_state=123, C=C_reg, solver='liblinear')
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_C = C_reg
    C_reg += 0.01

print('Лучший F1-score: {:.5f}'.format(best_score))

Лучший F1-score: 0.33390


Улучшения F1-score при изменении гиперпараметров для логистической регрессии отсутсвуют, поэтому в дальнейшем будем использовать стандартные настройки для этой модели.

- Дерево решений

Используем стандартные параметры:

In [39]:
model = DecisionTreeClassifier(random_state=123)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Decision Tree Default'
model_results.loc[i, 'F1-score'] = f1_score(target_valid, predictions_valid)
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1 score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1 score: 0.47375
AUC-ROC: 0.66663


Найдем наилучшую глубину дерева:

In [40]:
best_model = None
best_score = 0
for depth in range(2,51,1):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_depth = depth

model = DecisionTreeClassifier(random_state=123, max_depth=best_depth)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Decision Tree Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшая макс. глубина:', best_depth) 
print('F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))

Лучшая макс. глубина: 9
F1-score: 0.58136
AUC-ROC: 0.79680


- Случайный лес

Используем стандартные параметры:

In [41]:
model = RandomForestClassifier(random_state=123, n_estimators=10)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Random Forest Default'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1 score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1 score: 0.51682
AUC-ROC: 0.80216


Найдем наилучшее кол.-во оценщиков 'n_estimators' и глубину 'max_depth':

In [42]:
%%time
best_model = None
best_score = 0
best_est = 0
for est in range (40,101,20):
    for depth in range (9,20,2):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_model = model
            best_score = score
            best_est = est
            best_depth = depth

model = RandomForestClassifier(random_state=123, n_estimators=best_est, max_depth=best_depth)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)           

model_results.loc[i, 'model_name'] = 'Random Forest Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1 score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшее кол.-во оценщиков:', best_est)
print('Лучшая макс. глубина:', best_depth)

Лучшее значение F1 score: 0.60444
AUC-ROC: 0.84396
Лучшее кол.-во оценщиков: 80
Лучшая макс. глубина: 15
CPU times: user 15.7 s, sys: 0 ns, total: 15.7 s
Wall time: 15.7 s


### Вывод

На данном этапе был рассмотрен баланс отрицательного и положительного классов целевого признака - для тренировочной, валидационной и тестовой выборок он составляет примерно 80%/20% соответственно. Были созданы и обучены модели логистической регрессии, дерева решений, случайного леса на имеющейся несбалансированной выборке. Наилучший результат по получила модель случайного леса с гиперпараметрами n_estimators=80, max_depth=15 с результатами F1-score 0.60444, AUC-ROC: 0.84396. На втором месте находится модель дерева решений с макс. глубиной 9  с результатами F1-score: 0.58136, AUC-ROC: 0.79680. Наихудший результат показала модель логистической регрессии: F1 score: 0.33390, AUC-ROC: 0.75870. 

## 3. Борьба с дисбалансом <a id="step_3"></a>

Учитываем дисбаланс классов при помощи аргумента class_weight моделей.

- Логистическая регрессия

In [43]:
model = LogisticRegression(random_state=123, class_weight='balanced', solver='liblinear')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Logistic Regression Balanced'
model_results.loc[i, 'F1-score'] = f1
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.48889
AUC-ROC: 0.76358


- Дерево решений:

In [44]:
model = DecisionTreeClassifier(random_state=123, class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.46097
AUC-ROC: 0.65833


Найдем наилучшую глубину дерева:

In [45]:
best_model = None
best_score = 0
for depth in range(1,51,1):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_depth = depth

model = DecisionTreeClassifier(random_state=123, max_depth=best_depth, class_weight='balanced')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)         

model_results.loc[i, 'model_name'] = 'Decision Tree Balanced Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))

Лучшее значение F1-score: 0.59638
AUC-ROC: 0.83102


- Случайный лес:

In [46]:
model = RandomForestClassifier(random_state=123, class_weight='balanced', n_estimators=10)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.52761
AUC-ROC: 0.80212


Найдем наилучшее кол.-во оценщиков 'n_estimators' и глубину 'max_depth':

In [47]:
%%time
best_model = None
best_score = 0
best_est = 0
for est in range (20,40,2):
    for depth in range (2,15,2):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_model = model
            best_score = score
            best_est = est
            best_depth = depth

model = RandomForestClassifier(random_state=123, n_estimators=best_est, max_depth=best_depth, class_weight='balanced')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)  
            
model_results.loc[i, 'model_name'] = 'Random Forest Balanced Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшее кол.-во оценщиков:', best_est)
print('Лучшая макс. глубина:', best_depth)

Лучшее значение F1-score: 0.62959
AUC-ROC: 0.84469
Лучшее кол.-во оценщиков: 22
Лучшая макс. глубина: 10
CPU times: user 13.9 s, sys: 0 ns, total: 13.9 s
Wall time: 13.9 s


Используем функцию, добавлюящую записи в обучающие данные при помощи upsampling: 

In [48]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=123)
    
    return features_upsampled, target_upsampled

Вызываем функцию с коэффициентом увеличения выборки 4:

In [49]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Проверяем соотношение классов:

In [50]:
target_upsampled.value_counts(normalize=True)

False    0.501043
True     0.498957
Name: Exited, dtype: float64

Проверяем результат на различных моделях.

 - Логистическая регрессия:

In [51]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Logistic Regression Upsampled'
model_results.loc[i, 'F1-score'] = f1_score(target_valid, predictions_valid)
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.48889
AUC-ROC: 0.76348


- Дерево решений:

In [50]:
model = DecisionTreeClassifier(random_state=123)
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.45952
AUC-ROC: 0.65848


Находим наилучшую глубину дерева:

In [52]:
best_model = None
best_score = 0
for depth in range(1,51,1):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_depth = depth

model = DecisionTreeClassifier(random_state=123, max_depth=best_depth)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)          

model_results.loc[i, 'model_name'] = 'Decision Tree Upsampled Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшая макс. глубина:', best_depth)

Лучшее значение F1-score: 0.59638
AUC-ROC: 0.83102
Лучшая макс. глубина: 5


- Случайный лес:

In [53]:
model = RandomForestClassifier(random_state=123, n_estimators=10)
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))


F1-score: 0.48889
AUC-ROC: 0.82735


Находим наилучшее кол.-во оценщиков 'n_estimators' и глубину 'max_depth':

In [54]:
%%time
best_model = None
best_score = 0
best_est = 0
for est in range (1,101,20):
    for depth in range (1,20,2):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_model = model
            best_score = score
            best_est = est
            best_depth = depth

model = RandomForestClassifier(random_state=123, n_estimators=best_est, max_depth=best_depth)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)    

model_results.loc[i, 'model_name'] = 'Random Forest Upsampled Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшее кол.-во оценщиков:', best_est)
print('Лучшая макс. глубина:', best_depth)

Лучшее значение F1-score: 0.62433
AUC-ROC: 0.85162
Лучшее кол.-во оценщиков: 81
Лучшая макс. глубина: 9
CPU times: user 21.9 s, sys: 0 ns, total: 21.9 s
Wall time: 22 s


Уберем часть записей из обучающей выборки с помощью downsampling.

In [55]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

Вызываем функцию с коэффициентом 1/4:

In [56]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

Проверяем полученный баланс классов:

In [57]:
target_downsampled.value_counts(normalize=True)

False    0.501043
True     0.498957
Name: Exited, dtype: float64

Проверяем результат на различных моделях.

 - Логистическая регрессия:

In [58]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

model_results.loc[i, 'model_name'] = 'Logistic Regression Downsampled'
model_results.loc[i, 'F1-score'] = f1_score(target_valid, predictions_valid)
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.48966
AUC-ROC: 0.76321


- Дерево решений:

In [59]:
model = DecisionTreeClassifier(random_state=123)
model.fit(features_downsampled, target_downsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.47542
AUC-ROC: 0.68599


Находим наилучшую глубину дерева:

In [60]:
best_model = None
best_score = 0
for depth in range(1,51,1):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_downsampled, target_downsampled)
    predictions_valid = model.predict(features_valid)
    score = f1_score(target_valid, predictions_valid)
    if score > best_score:
        best_model = model
        best_score = score
        best_depth = depth
        
model = DecisionTreeClassifier(random_state=123, max_depth=best_depth)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)   

model_results.loc[i, 'model_name'] = 'Decision Tree Downsampled Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшая макс. глубина:', best_depth) 

Лучшее значение F1-score: 0.57567
AUC-ROC: 0.82842
Лучшая макс. глубина: 5


- Случайный лес:

In [61]:
model = RandomForestClassifier(random_state=123, n_estimators=10)
model.fit(features_downsampled, target_downsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.55067
AUC-ROC: 0.80810


Находим наилучшее кол.-во оценщиков 'n_estimators' и глубину 'max_depth':

In [62]:
%%time
best_model = None
best_score = 0
best_est = 0
for est in range (1,101,20):
    for depth in range (1,20,2):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_model = model
            best_score = score
            best_est = est
            best_depth = depth

model = RandomForestClassifier(random_state=123, n_estimators=best_est, max_depth=best_depth)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)  

model_results.loc[i, 'model_name'] = 'Random Forest Downsampled Optimized'
model_results.loc[i, 'F1-score'] = best_score
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

print('Лучшее значение F1-score: {:.5f}'.format(best_score))
print('AUC-ROC: {:.5f}'.format(auc_roc))
print('Лучшее кол.-во оценщиков:', best_est)
print('Лучшая макс. глубина:', best_depth)

Лучшее значение F1-score: 0.59738
AUC-ROC: 0.84561
Лучшее кол.-во оценщиков: 81
Лучшая макс. глубина: 13
CPU times: user 8.67 s, sys: 0 ns, total: 8.67 s
Wall time: 8.67 s


Далее изучим влияние порогового значения отнесения значения к классу "0" или "1".

- Логистическая регрессия

In [63]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.2, 0.4, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, F1-score = {:.3f}".format(
        threshold, precision, recall, f1, auc_roc))

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC: {:.5f}'.format(auc_roc))

Порог = 0.20 | Точность = 0.371, Полнота = 0.675, F1-score = 0.479
Порог = 0.21 | Точность = 0.380, Полнота = 0.656, F1-score = 0.481
Порог = 0.22 | Точность = 0.389, Полнота = 0.639, F1-score = 0.484
Порог = 0.23 | Точность = 0.405, Полнота = 0.629, F1-score = 0.493
Порог = 0.24 | Точность = 0.412, Полнота = 0.615, F1-score = 0.493
Порог = 0.25 | Точность = 0.422, Полнота = 0.603, F1-score = 0.497
Порог = 0.26 | Точность = 0.434, Полнота = 0.589, F1-score = 0.499
Порог = 0.27 | Точность = 0.443, Полнота = 0.574, F1-score = 0.500
Порог = 0.28 | Точность = 0.436, Полнота = 0.550, F1-score = 0.487
Порог = 0.29 | Точность = 0.446, Полнота = 0.538, F1-score = 0.488
Порог = 0.30 | Точность = 0.458, Полнота = 0.522, F1-score = 0.488
Порог = 0.31 | Точность = 0.465, Полнота = 0.498, F1-score = 0.481
Порог = 0.32 | Точность = 0.464, Полнота = 0.469, F1-score = 0.467
Порог = 0.33 | Точность = 0.474, Полнота = 0.455, F1-score = 0.464
Порог = 0.34 | Точность = 0.483, Полнота = 0.440, F1-score = 0

Наилучшее значением F1-score получено при пороге 0.27.

In [64]:
model_results.loc[i, 'model_name'] = 'Logistic Regression Threshold=0.27'
model_results.loc[i, 'F1-score'] = 0.5
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

- Дерево решений

In [65]:
model = DecisionTreeClassifier(random_state=123, max_depth=5)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.2, 0.4, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, F1-score = {:.3f}".format(
        threshold, precision, recall, f1))
    
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC: {:.5f}'.format(auc_roc))  

Порог = 0.20 | Точность = 0.465, Полнота = 0.708, F1-score = 0.561
Порог = 0.21 | Точность = 0.465, Полнота = 0.708, F1-score = 0.561
Порог = 0.22 | Точность = 0.465, Полнота = 0.708, F1-score = 0.561
Порог = 0.23 | Точность = 0.541, Полнота = 0.653, F1-score = 0.592
Порог = 0.24 | Точность = 0.630, Полнота = 0.555, F1-score = 0.590
Порог = 0.25 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.26 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.27 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.28 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.29 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.30 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.31 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.32 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.33 | Точность = 0.632, Полнота = 0.555, F1-score = 0.591
Порог = 0.34 | Точность = 0.720, Полнота = 0.493, F1-score = 0

Наилучшее значением F1-score получено при пороге 0.3.

In [66]:
model_results.loc[i, 'model_name'] = 'Decision Tree Threshold=0.3'
model_results.loc[i, 'F1-score'] = 0.5
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

- Случайный лес

In [67]:
model = RandomForestClassifier(random_state=123, n_estimators=22, max_depth=10)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.2, 0.4, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, F1-score = {:.3f}".format(
        threshold, precision, recall, f1))
    
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC: {:.5f}'.format(auc_roc))

Порог = 0.20 | Точность = 0.471, Полнота = 0.749, F1-score = 0.579
Порог = 0.21 | Точность = 0.486, Полнота = 0.737, F1-score = 0.586
Порог = 0.22 | Точность = 0.509, Полнота = 0.727, F1-score = 0.599
Порог = 0.23 | Точность = 0.519, Полнота = 0.713, F1-score = 0.601
Порог = 0.24 | Точность = 0.530, Полнота = 0.708, F1-score = 0.606
Порог = 0.25 | Точность = 0.549, Полнота = 0.701, F1-score = 0.616
Порог = 0.26 | Точность = 0.571, Полнота = 0.694, F1-score = 0.626
Порог = 0.27 | Точность = 0.579, Полнота = 0.682, F1-score = 0.626
Порог = 0.28 | Точность = 0.600, Полнота = 0.672, F1-score = 0.634
Порог = 0.29 | Точность = 0.616, Полнота = 0.660, F1-score = 0.637
Порог = 0.30 | Точность = 0.624, Полнота = 0.648, F1-score = 0.636
Порог = 0.31 | Точность = 0.634, Полнота = 0.641, F1-score = 0.637
Порог = 0.32 | Точность = 0.644, Полнота = 0.624, F1-score = 0.634
Порог = 0.33 | Точность = 0.651, Полнота = 0.608, F1-score = 0.629
Порог = 0.34 | Точность = 0.655, Полнота = 0.591, F1-score = 0

Наилучшее значением F1-score получено при пороге 0.29.

In [68]:
model_results.loc[i, 'model_name'] = 'Random Forest Threshold=0.29'
model_results.loc[i, 'F1-score'] = 0.637
model_results.loc[i, 'AUC-ROC'] = auc_roc
i+=1

Выведем полученную таблицу с результатами метрик и отсортируем по убыванию F1-score:

In [69]:
model_results.sort_values(by='F1-score', ascending=False).reset_index(drop=True)

,model_name,F1-score,AUC-ROC
0,Random Forest Threshold=0.29,0.637,0.847426
1,Random Forest Balanced Optimized,0.629586,0.844691
2,Random Forest Upsampled Optimized,0.624327,0.85162
3,Random Forest Optimized,0.604444,0.843963
4,Random Forest Downsampled Optimized,0.597378,0.845611
5,Decision Tree Balanced Optimized,0.596379,0.831024
6,Decision Tree Upsampled Optimized,0.596379,0.831024
7,Decision Tree Optimized,0.581363,0.7968
8,Random Forest Default,0.581363,0.802157
9,Decision Tree Downsampled Optimized,0.575673,0.828423


### Вывод

На данном этапе были обучены модели с учетом дисбаланса классов. Были обучены модели с аргументом class_weight='balanced'. Также модели обучены на выборках, увеличенных при помощи upsampling, downsampling. Было рассмотрено влияние порогового значения вероятности отнесения целевого признака к положительному/отрицательному классам. Результаты метрик полученных моделей занесены в таблицу и выведены в порядке убывания F1-score. В результате выяснено, что наилучший результат имеет модель случайного леса с пороговым значением 0.29: F1-score 0.637, AUC-ROC 0.847426. Высокое значение также AUC говорит о том, что модель с высокой долей вероятности сможет успешно разделить классы целевого признака. На втором месте находится модель случайного леса со сбалансированной выборкой, 22 оценщиками и глубиной 10: F1-score 0.629586, AUC-ROC 0.844691. На третьем месте находится модель случайного леса со сбалансированной при помощи апсемплинга выборкой, 81 оценщиком и глубиной 9: F1-score 0.624327, AUC-ROC 0.85162 (макс. полученный результат). Самый худший результат показала логистическая регрессия с несбалансированной выборкой и параметрами по-умолчанию: F1-score 0.333895, AUC-ROC 0.758632.

## 4. Тестирование модели <a id="step_4"></a>

Для обучения используем модель с наилучшим показателем F1-score - случайный лес с кол.-вом оценщиков 80, макс. глубиной 15 и пороговым значением 0.29:

In [69]:
model = RandomForestClassifier(random_state=123, n_estimators=80, max_depth=15)
model.fit(features_train, target_train)

threshold = 0.29
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > threshold
f1 = f1_score(target_test, predicted_test)
auc_roc = roc_auc_score(target_test, probabilities_one_test)

print('F1-score: {:.5f}'.format(f1))
print('AUC-ROC: {:.5f}'.format(auc_roc))

F1-score: 0.62115
AUC-ROC: 0.85551


### Вывод

На данном этапе была использована модель с наилучшим показателем F1-score, полученным на предыдущем этапе. В результате получен массив целевых признаков (predicted_test) с показателем F1-score 0.62115, AUC-ROC 0.85551. 

## Чек-лист готовности проекта <a id="checklist"></a>

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Исследована метрика *AUC-ROC*